# Hybrid Frameworks for Predictive Maintenance

This notebook implements **state-of-the-art hybrid approaches** combining multiple techniques:

1. **FNO + Physics-Informed** - Fourier Neural Operators with physical constraints
2. **Ensemble Deep Learning** - VAE + LSTM + Transformer combination
3. **Neural Network + Reinforcement Learning** - RUL prediction with maintenance scheduling
4. **Multi-Modal Fusion** - Combining different sensor modalities

## Why Hybrid Approaches?

| Advantage | Description |
|-----------|-------------|
| **Best of Multiple Worlds** | Combine strengths of different architectures |
| **Physics + Data-Driven** | Incorporate domain knowledge with learning |
| **Robust Predictions** | Ensemble methods reduce variance |
| **End-to-End Optimization** | Prediction + Decision in one pipeline |

## Recent Research Trends (2025)

- Hybrid CNN-LSTM + Attention: ~96% accuracy on failure prediction
- Ensemble VAE + LSTM + Transformer: 48h early warning for wind turbines
- FNO + DAE + GNN + RL: ~13% cost reduction vs baselines

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import seaborn as sns
import os
import json

np.random.seed(42)

# Check TensorFlow
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    print(f"TensorFlow {tf.__version__} available")
    HAS_TF = True
except ImportError:
    print("TensorFlow not available")
    HAS_TF = False

# Directories
DATA_DIR = '../data/hybrid'
MODEL_DIR = '../models/hybrid'
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

print("Setup complete!")

## 1. Fourier Neural Operator (FNO) for Time Series

FNO operates in the frequency domain, making it efficient for:
- Periodic signals (rotating machinery)
- Physical systems with known frequency characteristics
- Resolution-independent predictions

In [ ]:
if HAS_TF:
    
    class SpectralConv1D(layers.Layer):
        """
        1D Spectral Convolution Layer (core of FNO).
        
        Performs convolution in Fourier space for global receptive field.
        """
        
        def __init__(self, out_channels, modes, **kwargs):
            super().__init__(**kwargs)
            self.out_channels = out_channels
            self.modes = modes  # Number of Fourier modes to keep
            
        def build(self, input_shape):
            in_channels = input_shape[-1]
            
            # Complex-valued weights for Fourier modes
            self.weights_real = self.add_weight(
                name='weights_real',
                shape=(in_channels, self.out_channels, self.modes),
                initializer='glorot_uniform',
                trainable=True
            )
            self.weights_imag = self.add_weight(
                name='weights_imag',
                shape=(in_channels, self.out_channels, self.modes),
                initializer='glorot_uniform',
                trainable=True
            )
            
            super().build(input_shape)
            
        def call(self, inputs):
            """
            Args:
                inputs: [batch, seq_len, in_channels]
                
            Returns:
                output: [batch, seq_len, out_channels]
            """
            batch_size = tf.shape(inputs)[0]
            seq_len = tf.shape(inputs)[1]
            
            # FFT along sequence dimension
            # [batch, seq, channels] -> transpose -> FFT
            x = tf.transpose(inputs, [0, 2, 1])  # [batch, channels, seq]
            x_ft = tf.signal.rfft(tf.cast(x, tf.float32))  # [batch, channels, freq]
            
            # Keep only low frequency modes
            x_ft = x_ft[:, :, :self.modes]
            
            # Complex multiplication in Fourier space
            weights = tf.complex(self.weights_real, self.weights_imag)
            # [batch, in_ch, modes] x [in_ch, out_ch, modes] -> [batch, out_ch, modes]
            out_ft = tf.einsum('bim,iom->bom', x_ft, weights)
            
            # Pad to original size and inverse FFT
            n_freq = seq_len // 2 + 1
            paddings = [[0, 0], [0, 0], [0, n_freq - self.modes]]
            out_ft = tf.pad(out_ft, paddings)
            
            # Inverse FFT
            out = tf.signal.irfft(out_ft, fft_length=[seq_len])
            
            # Transpose back
            out = tf.transpose(out, [0, 2, 1])  # [batch, seq, out_channels]
            
            return out
        
        def get_config(self):
            config = super().get_config()
            config.update({'out_channels': self.out_channels, 'modes': self.modes})
            return config
    
    
    class FNOBlock(layers.Layer):
        """
        Complete FNO block with spectral conv + skip connection.
        """
        
        def __init__(self, channels, modes, **kwargs):
            super().__init__(**kwargs)
            self.channels = channels
            self.modes = modes
            
        def build(self, input_shape):
            self.spectral_conv = SpectralConv1D(self.channels, self.modes)
            self.linear = layers.Dense(self.channels)
            self.norm = layers.LayerNormalization()
            
            super().build(input_shape)
            
        def call(self, inputs):
            # Spectral pathway
            x1 = self.spectral_conv(inputs)
            
            # Linear pathway (skip)
            x2 = self.linear(inputs)
            
            # Combine
            x = x1 + x2
            x = self.norm(x)
            x = tf.nn.gelu(x)
            
            return x
        
        def get_config(self):
            config = super().get_config()
            config.update({'channels': self.channels, 'modes': self.modes})
            return config
    
    print("FNO layers defined")

In [ ]:
if HAS_TF:
    
    def build_physics_informed_fno(
        seq_length,
        n_features,
        n_modes=16,
        width=64,
        n_layers=4
    ):
        """
        Physics-Informed Fourier Neural Operator.
        
        Combines data-driven learning with physical constraints.
        """
        inputs = keras.Input(shape=(seq_length, n_features))
        
        # Lift to higher dimension
        x = layers.Dense(width)(inputs)
        
        # FNO layers
        for i in range(n_layers):
            x = FNOBlock(width, n_modes, name=f'fno_block_{i}')(x)
        
        # Project down
        x = layers.Dense(width // 2, activation='gelu')(x)
        
        # Two outputs:
        # 1. RUL prediction
        rul_out = layers.GlobalAveragePooling1D()(x)
        rul_out = layers.Dense(32, activation='relu')(rul_out)
        rul_out = layers.Dense(1, activation='relu', name='rul')(rul_out)
        
        # 2. Health index (sequence output for monitoring)
        health_out = layers.Dense(1, activation='sigmoid', name='health')(x)
        health_out = tf.squeeze(health_out, axis=-1)
        
        model = keras.Model(
            inputs=inputs,
            outputs={'rul': rul_out, 'health': health_out}
        )
        return model
    
    print("Physics-Informed FNO builder defined")

## 2. Ensemble Deep Learning

Combine multiple architectures for robust predictions.

In [ ]:
if HAS_TF:
    
    class VariationalEncoder(layers.Layer):
        """
        Variational Autoencoder encoder for learning latent representations.
        """
        
        def __init__(self, latent_dim, **kwargs):
            super().__init__(**kwargs)
            self.latent_dim = latent_dim
            
        def build(self, input_shape):
            self.conv1 = layers.Conv1D(32, 5, strides=2, padding='same', activation='relu')
            self.conv2 = layers.Conv1D(64, 3, strides=2, padding='same', activation='relu')
            self.flatten = layers.GlobalAveragePooling1D()
            self.dense = layers.Dense(64, activation='relu')
            self.z_mean = layers.Dense(self.latent_dim)
            self.z_log_var = layers.Dense(self.latent_dim)
            
            super().build(input_shape)
            
        def call(self, inputs, training=None):
            x = self.conv1(inputs)
            x = self.conv2(x)
            x = self.flatten(x)
            x = self.dense(x)
            
            z_mean = self.z_mean(x)
            z_log_var = self.z_log_var(x)
            
            # Reparameterization trick
            if training:
                epsilon = tf.random.normal(tf.shape(z_mean))
                z = z_mean + tf.exp(0.5 * z_log_var) * epsilon
            else:
                z = z_mean
            
            return z, z_mean, z_log_var
        
        def get_config(self):
            config = super().get_config()
            config.update({'latent_dim': self.latent_dim})
            return config
    
    print("VariationalEncoder defined")

In [ ]:
if HAS_TF:
    
    def build_ensemble_model(seq_length, n_features, n_classes=None):
        """
        Ensemble model combining VAE, LSTM, and Transformer.
        
        Each component captures different aspects:
        - VAE: Latent representation + uncertainty
        - LSTM: Sequential dependencies
        - Transformer: Long-range attention patterns
        """
        inputs = keras.Input(shape=(seq_length, n_features))
        
        # Branch 1: VAE for latent features
        vae_encoder = VariationalEncoder(latent_dim=16)
        z, z_mean, z_log_var = vae_encoder(inputs)
        vae_features = z  # [batch, 16]
        
        # Branch 2: LSTM for sequential patterns
        lstm_out = layers.LSTM(64, return_sequences=True)(inputs)
        lstm_out = layers.LSTM(32)(lstm_out)
        lstm_features = lstm_out  # [batch, 32]
        
        # Branch 3: Transformer for attention
        # Simple multi-head attention
        attn_out = layers.MultiHeadAttention(
            num_heads=4, key_dim=16
        )(inputs, inputs)
        attn_out = layers.GlobalAveragePooling1D()(attn_out)
        transformer_features = layers.Dense(32, activation='relu')(attn_out)  # [batch, 32]
        
        # Fusion: Concatenate all branches
        fused = layers.Concatenate()([vae_features, lstm_features, transformer_features])
        # [batch, 16 + 32 + 32 = 80]
        
        # Final layers
        x = layers.Dense(64, activation='relu')(fused)
        x = layers.Dropout(0.3)(x)
        x = layers.Dense(32, activation='relu')(x)
        
        if n_classes:
            # Classification
            outputs = layers.Dense(n_classes, activation='softmax')(x)
            loss = 'sparse_categorical_crossentropy'
        else:
            # Regression (RUL)
            outputs = layers.Dense(1, activation='relu')(x)
            loss = 'mse'
        
        model = keras.Model(inputs=inputs, outputs=outputs)
        
        # Add KL divergence loss for VAE regularization
        kl_loss = -0.5 * tf.reduce_mean(
            1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
        )
        model.add_loss(0.001 * kl_loss)  # Small weight for KL term
        
        return model
    
    print("Ensemble model builder defined")

## 3. Generate Training Data

In [ ]:
def generate_physics_based_data(n_samples=500, seq_length=128, n_features=6):
    """
    Generate data with realistic physics-based degradation.
    
    Models:
    - Vibration: f(speed, load, wear)
    - Temperature: f(friction, cooling, ambient)
    - Current: f(load, resistance increase)
    """
    X = []
    y_rul = []
    y_health = []
    
    for _ in range(n_samples):
        # Random machine parameters
        total_life = np.random.randint(200, 500)
        current_age = np.random.randint(0, total_life)
        
        t = np.arange(seq_length)
        
        # Normalized time in lifecycle
        lifecycle_pos = (current_age + t / seq_length * 50) / total_life
        lifecycle_pos = np.clip(lifecycle_pos, 0, 1.2)  # Allow slight overflow
        
        # Wear factor (nonlinear degradation)
        wear = 0.1 + 0.9 * (lifecycle_pos ** 2)
        
        features = np.zeros((seq_length, n_features))
        
        # Operating conditions
        speed = 1500 + np.random.normal(0, 50)  # RPM
        load = 0.6 + 0.2 * np.random.random()  # Load factor
        
        # Feature 0: Vibration amplitude (increases with wear)
        f_rot = speed / 60
        base_vib = 0.5 + 2.0 * wear
        features[:, 0] = base_vib * np.sin(2 * np.pi * f_rot * t / 100)
        features[:, 0] += np.random.normal(0, 0.1 * wear, seq_length)
        
        # Feature 1: 2x vibration (misalignment indicator)
        features[:, 1] = 0.3 * base_vib * np.sin(4 * np.pi * f_rot * t / 100)
        features[:, 1] += np.random.normal(0, 0.05, seq_length)
        
        # Feature 2: Temperature (thermal model)
        friction_heat = 20 * wear * load
        ambient = 25
        cooling_efficiency = 1 - 0.3 * wear
        temp = ambient + friction_heat / cooling_efficiency
        features[:, 2] = temp + np.random.normal(0, 2, seq_length)
        features[:, 2] += 5 * np.sin(0.1 * t)  # Thermal cycling
        
        # Feature 3: Motor current (electrical model)
        base_current = 10 * load
        resistance_increase = 1 + 0.2 * wear
        features[:, 3] = base_current * resistance_increase
        features[:, 3] += np.random.normal(0, 0.3, seq_length)
        
        # Feature 4: Oil pressure (decreases with wear)
        features[:, 4] = 5.0 * (1 - 0.4 * wear)
        features[:, 4] += np.random.normal(0, 0.1, seq_length)
        
        # Feature 5: Acoustic emission (high frequency content)
        features[:, 5] = 0.3 + 1.5 * wear ** 1.5
        features[:, 5] += np.random.normal(0, 0.2, seq_length)
        
        # RUL at end of sequence
        rul = max(0, total_life - current_age - 50)
        
        # Health index (1 = healthy, 0 = failed)
        health = np.maximum(0, 1 - lifecycle_pos)
        
        X.append(features)
        y_rul.append(rul)
        y_health.append(health)
    
    return np.array(X), np.array(y_rul), np.array(y_health)

# Generate data
print("Generating physics-based degradation data...")
X_physics, y_rul, y_health = generate_physics_based_data(n_samples=1000)
print(f"Generated: X={X_physics.shape}, RUL={y_rul.shape}")

In [ ]:
# Visualize physics-based features
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
feature_names = ['Vibration 1x', 'Vibration 2x', 'Temperature', 'Current', 'Oil Pressure', 'Acoustic']

# Show samples with different RUL values
rul_bins = [0, 100, 200, 300]

for i, ax in enumerate(axes.flat):
    for rul_threshold in rul_bins:
        mask = np.abs(y_rul - rul_threshold) < 30
        if mask.any():
            idx = np.where(mask)[0][0]
            ax.plot(X_physics[idx, :, i], alpha=0.7, label=f'RUL≈{rul_threshold}')
    ax.set_title(feature_names[i])
    ax.set_xlabel('Time')
    if i == 0:
        ax.legend(fontsize=8)

plt.suptitle('Physics-Based Sensor Features at Different RUL Stages')
plt.tight_layout()
plt.savefig(f'{DATA_DIR}/physics_features.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Train FNO Model

In [ ]:
if HAS_TF:
    # Normalize data
    scaler = StandardScaler()
    X_flat = X_physics.reshape(-1, X_physics.shape[-1])
    scaler.fit(X_flat)
    X_scaled = np.array([scaler.transform(x) for x in X_physics])
    
    # Normalize RUL
    rul_max = y_rul.max()
    y_rul_norm = y_rul / rul_max
    
    # Split
    X_train, X_test, y_rul_train, y_rul_test, y_health_train, y_health_test = train_test_split(
        X_scaled, y_rul_norm, y_health, test_size=0.2, random_state=42
    )
    
    print(f"Training: {X_train.shape}")
    print(f"Test: {X_test.shape}")

In [ ]:
if HAS_TF:
    # Build FNO model
    fno_model = build_physics_informed_fno(
        seq_length=128,
        n_features=6,
        n_modes=16,
        width=64,
        n_layers=4
    )
    
    fno_model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss={
            'rul': 'mse',
            'health': 'mse'
        },
        loss_weights={'rul': 1.0, 'health': 0.5},
        metrics={'rul': 'mae'}
    )
    
    fno_model.summary()

In [ ]:
if HAS_TF:
    print("Training FNO model...")
    fno_history = fno_model.fit(
        X_train,
        {'rul': y_rul_train, 'health': y_health_train},
        validation_split=0.15,
        epochs=50,
        batch_size=32,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
        ],
        verbose=1
    )

In [ ]:
if HAS_TF:
    # Evaluate FNO
    predictions = fno_model.predict(X_test)
    y_pred_rul = predictions['rul'].flatten() * rul_max
    y_test_rul_orig = y_rul_test * rul_max
    
    rmse = np.sqrt(mean_squared_error(y_test_rul_orig, y_pred_rul))
    mae = mean_absolute_error(y_test_rul_orig, y_pred_rul)
    r2 = r2_score(y_test_rul_orig, y_pred_rul)
    
    print(f"\nFNO RUL Prediction Results:")
    print(f"  RMSE: {rmse:.2f} cycles")
    print(f"  MAE:  {mae:.2f} cycles")
    print(f"  R²:   {r2:.4f}")
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Training history
    axes[0].plot(fno_history.history['loss'], label='Train')
    axes[0].plot(fno_history.history['val_loss'], label='Val')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('FNO Training')
    axes[0].legend()
    
    # RUL prediction
    axes[1].scatter(y_test_rul_orig, y_pred_rul, alpha=0.5)
    axes[1].plot([0, rul_max], [0, rul_max], 'r--', label='Perfect')
    axes[1].set_xlabel('Actual RUL')
    axes[1].set_ylabel('Predicted RUL')
    axes[1].set_title(f'FNO RUL (R²={r2:.3f})')
    axes[1].legend()
    
    # Health index example
    idx = 0
    axes[2].plot(y_health_test[idx], label='Actual', linewidth=2)
    axes[2].plot(predictions['health'][idx], label='Predicted', linewidth=2, alpha=0.7)
    axes[2].set_xlabel('Time Step')
    axes[2].set_ylabel('Health Index')
    axes[2].set_title('Health Index Prediction')
    axes[2].legend()
    
    plt.tight_layout()
    plt.savefig(f'{MODEL_DIR}/fno_results.png', dpi=150, bbox_inches='tight')
    plt.show()

## 5. Train Ensemble Model

In [ ]:
if HAS_TF:
    # Build ensemble model
    ensemble_model = build_ensemble_model(
        seq_length=128,
        n_features=6,
        n_classes=None  # Regression mode
    )
    
    ensemble_model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss='mse',
        metrics=['mae']
    )
    
    ensemble_model.summary()

In [ ]:
if HAS_TF:
    print("Training Ensemble model...")
    ensemble_history = ensemble_model.fit(
        X_train, y_rul_train,
        validation_split=0.15,
        epochs=50,
        batch_size=32,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
        ],
        verbose=1
    )

In [ ]:
if HAS_TF:
    # Evaluate ensemble
    y_pred_ensemble = ensemble_model.predict(X_test).flatten() * rul_max
    
    rmse_ens = np.sqrt(mean_squared_error(y_test_rul_orig, y_pred_ensemble))
    mae_ens = mean_absolute_error(y_test_rul_orig, y_pred_ensemble)
    r2_ens = r2_score(y_test_rul_orig, y_pred_ensemble)
    
    print(f"\nEnsemble RUL Prediction Results:")
    print(f"  RMSE: {rmse_ens:.2f} cycles")
    print(f"  MAE:  {mae_ens:.2f} cycles")
    print(f"  R²:   {r2_ens:.4f}")

## 6. Reinforcement Learning for Maintenance Scheduling

Use RL to decide optimal maintenance timing based on RUL predictions.

In [ ]:
class MaintenanceEnvironment:
    """
    Simulated maintenance scheduling environment.
    
    State: [current_health, predicted_rul, time_since_maintenance, production_demand]
    Actions: 0 = continue, 1 = preventive maintenance, 2 = defer to next window
    Rewards: Balance maintenance cost vs failure cost
    """
    
    def __init__(self):
        self.state_dim = 4
        self.action_dim = 3
        self.reset()
        
        # Cost parameters
        self.preventive_cost = 100
        self.failure_cost = 1000
        self.production_value = 10  # per time step
        self.downtime_preventive = 2
        self.downtime_failure = 10
        
    def reset(self):
        self.health = 1.0
        self.true_rul = np.random.randint(50, 200)
        self.time_since_maintenance = 0
        self.production_demand = np.random.uniform(0.5, 1.0)
        self.time = 0
        self.done = False
        return self._get_state()
    
    def _get_state(self):
        # Predicted RUL has some noise
        predicted_rul = max(0, self.true_rul + np.random.normal(0, 10))
        return np.array([
            self.health,
            predicted_rul / 200,  # Normalized
            min(self.time_since_maintenance / 100, 1.0),
            self.production_demand
        ])
    
    def step(self, action):
        reward = 0
        info = {}
        
        if action == 1:  # Preventive maintenance
            reward -= self.preventive_cost
            reward -= self.downtime_preventive * self.production_value * self.production_demand
            self.health = 1.0
            self.true_rul = np.random.randint(150, 200)
            self.time_since_maintenance = 0
            info['action'] = 'preventive'
            
        elif action == 2:  # Defer
            # Small penalty for deferring when health is low
            if self.health < 0.3:
                reward -= 20
            info['action'] = 'defer'
            
        else:  # Continue
            reward += self.production_value * self.production_demand
            info['action'] = 'continue'
        
        # Time passes
        self.time += 1
        self.time_since_maintenance += 1
        self.true_rul -= 1
        
        # Health degrades (with noise)
        degradation = 0.01 + 0.02 * (1 - self.health)
        self.health = max(0, self.health - degradation + np.random.normal(0, 0.005))
        
        # Check for failure
        if self.true_rul <= 0 or self.health <= 0:
            reward -= self.failure_cost
            reward -= self.downtime_failure * self.production_value * self.production_demand
            self.done = True
            info['failure'] = True
        
        # Update demand (varies over time)
        self.production_demand = np.clip(
            self.production_demand + np.random.normal(0, 0.1),
            0.3, 1.0
        )
        
        # Episode ends after 200 steps or failure
        if self.time >= 200:
            self.done = True
        
        return self._get_state(), reward, self.done, info

print("MaintenanceEnvironment defined")

In [ ]:
if HAS_TF:
    
    class DQNAgent:
        """
        Simple DQN agent for maintenance scheduling.
        """
        
        def __init__(self, state_dim, action_dim):
            self.state_dim = state_dim
            self.action_dim = action_dim
            self.memory = []
            self.gamma = 0.95
            self.epsilon = 1.0
            self.epsilon_min = 0.01
            self.epsilon_decay = 0.995
            
            self.model = self._build_model()
            self.target_model = self._build_model()
            self.update_target()
            
        def _build_model(self):
            model = keras.Sequential([
                layers.Dense(64, activation='relu', input_shape=(self.state_dim,)),
                layers.Dense(64, activation='relu'),
                layers.Dense(32, activation='relu'),
                layers.Dense(self.action_dim, activation='linear')
            ])
            model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='mse')
            return model
        
        def update_target(self):
            self.target_model.set_weights(self.model.get_weights())
            
        def remember(self, state, action, reward, next_state, done):
            self.memory.append((state, action, reward, next_state, done))
            if len(self.memory) > 10000:
                self.memory.pop(0)
                
        def act(self, state, training=True):
            if training and np.random.random() < self.epsilon:
                return np.random.randint(self.action_dim)
            q_values = self.model.predict(state[np.newaxis], verbose=0)
            return np.argmax(q_values[0])
        
        def replay(self, batch_size=32):
            if len(self.memory) < batch_size:
                return
            
            indices = np.random.choice(len(self.memory), batch_size, replace=False)
            batch = [self.memory[i] for i in indices]
            
            states = np.array([b[0] for b in batch])
            actions = np.array([b[1] for b in batch])
            rewards = np.array([b[2] for b in batch])
            next_states = np.array([b[3] for b in batch])
            dones = np.array([b[4] for b in batch])
            
            # Q-learning update
            target_q = self.model.predict(states, verbose=0)
            next_q = self.target_model.predict(next_states, verbose=0)
            
            for i in range(batch_size):
                if dones[i]:
                    target_q[i, actions[i]] = rewards[i]
                else:
                    target_q[i, actions[i]] = rewards[i] + self.gamma * np.max(next_q[i])
            
            self.model.fit(states, target_q, epochs=1, verbose=0)
            
            # Decay epsilon
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay
    
    print("DQNAgent defined")

In [ ]:
if HAS_TF:
    # Train RL agent
    env = MaintenanceEnvironment()
    agent = DQNAgent(env.state_dim, env.action_dim)
    
    n_episodes = 200
    rewards_history = []
    failures_history = []
    
    print("Training RL maintenance scheduler...")
    
    for episode in range(n_episodes):
        state = env.reset()
        total_reward = 0
        failed = False
        
        while not env.done:
            action = agent.act(state)
            next_state, reward, done, info = env.step(action)
            agent.remember(state, action, reward, next_state, done)
            agent.replay()
            
            state = next_state
            total_reward += reward
            
            if info.get('failure'):
                failed = True
        
        rewards_history.append(total_reward)
        failures_history.append(1 if failed else 0)
        
        # Update target network periodically
        if episode % 10 == 0:
            agent.update_target()
            print(f"Episode {episode}, Reward: {total_reward:.0f}, "
                  f"Failures: {sum(failures_history[-20:])}/20, "
                  f"ε: {agent.epsilon:.3f}")

In [ ]:
if HAS_TF:
    # Visualize RL training
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Rewards
    window = 20
    smoothed_rewards = np.convolve(rewards_history, np.ones(window)/window, mode='valid')
    axes[0].plot(smoothed_rewards)
    axes[0].set_xlabel('Episode')
    axes[0].set_ylabel('Total Reward')
    axes[0].set_title('Training Rewards (Smoothed)')
    
    # Failure rate
    failure_rate = np.convolve(failures_history, np.ones(window)/window, mode='valid')
    axes[1].plot(failure_rate)
    axes[1].set_xlabel('Episode')
    axes[1].set_ylabel('Failure Rate')
    axes[1].set_title('Failure Rate (20-episode window)')
    
    # Test the trained policy
    test_episodes = 50
    test_rewards = []
    test_failures = 0
    maintenance_actions = []
    
    for _ in range(test_episodes):
        state = env.reset()
        ep_reward = 0
        while not env.done:
            action = agent.act(state, training=False)
            if action == 1:
                maintenance_actions.append(state[0])  # Health at maintenance
            state, reward, done, info = env.step(action)
            ep_reward += reward
            if info.get('failure'):
                test_failures += 1
        test_rewards.append(ep_reward)
    
    # Health at maintenance decision
    if maintenance_actions:
        axes[2].hist(maintenance_actions, bins=20, edgecolor='black')
        axes[2].set_xlabel('Health Level')
        axes[2].set_ylabel('Count')
        axes[2].set_title('Health at Maintenance Decision')
        axes[2].axvline(np.mean(maintenance_actions), color='r', linestyle='--', 
                       label=f'Mean: {np.mean(maintenance_actions):.2f}')
        axes[2].legend()
    
    plt.tight_layout()
    plt.savefig(f'{MODEL_DIR}/rl_training.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\nTest Results (50 episodes):")
    print(f"  Average Reward: {np.mean(test_rewards):.0f}")
    print(f"  Failure Rate: {test_failures/test_episodes*100:.1f}%")
    if maintenance_actions:
        print(f"  Avg Health at Maintenance: {np.mean(maintenance_actions):.2f}")

## 7. Multi-Modal Fusion

Combine different sensor types with specialized processing.

In [ ]:
if HAS_TF:
    
    def build_multimodal_model(
        vibration_shape,  # [time, channels]
        thermal_shape,    # [time, channels]
        electrical_shape, # [time, channels]
        n_outputs=1
    ):
        """
        Multi-modal fusion model for different sensor types.
        
        Each modality has specialized processing:
        - Vibration: Frequency analysis (FNO-style)
        - Thermal: Trend analysis (LSTM)
        - Electrical: Pattern matching (CNN)
        """
        
        # Vibration branch (frequency-focused)
        vib_input = keras.Input(shape=vibration_shape, name='vibration')
        vib = FNOBlock(32, modes=8)(vib_input)
        vib = layers.GlobalAveragePooling1D()(vib)
        vib = layers.Dense(32, activation='relu')(vib)
        
        # Thermal branch (trend-focused)
        therm_input = keras.Input(shape=thermal_shape, name='thermal')
        therm = layers.LSTM(32, return_sequences=True)(therm_input)
        therm = layers.LSTM(16)(therm)
        therm = layers.Dense(32, activation='relu')(therm)
        
        # Electrical branch (pattern-focused)
        elec_input = keras.Input(shape=electrical_shape, name='electrical')
        elec = layers.Conv1D(32, 5, activation='relu', padding='same')(elec_input)
        elec = layers.Conv1D(64, 3, activation='relu', padding='same')(elec)
        elec = layers.GlobalAveragePooling1D()(elec)
        elec = layers.Dense(32, activation='relu')(elec)
        
        # Cross-modal attention
        # Stack modality features
        modalities = tf.stack([vib, therm, elec], axis=1)  # [batch, 3, 32]
        
        # Self-attention across modalities
        attn = layers.MultiHeadAttention(num_heads=2, key_dim=16)(modalities, modalities)
        attn = layers.GlobalAveragePooling1D()(attn)
        
        # Fusion
        fused = layers.Concatenate()([vib, therm, elec, attn])
        fused = layers.Dense(64, activation='relu')(fused)
        fused = layers.Dropout(0.3)(fused)
        fused = layers.Dense(32, activation='relu')(fused)
        
        output = layers.Dense(n_outputs, activation='relu')(fused)
        
        model = keras.Model(
            inputs=[vib_input, therm_input, elec_input],
            outputs=output
        )
        return model
    
    print("Multi-modal fusion model defined")

In [ ]:
# Generate multi-modal data
def generate_multimodal_data(n_samples=500, seq_length=128):
    """
    Generate data from different sensor modalities.
    """
    vibration = []  # 2 channels: axial, radial
    thermal = []    # 3 channels: bearing, motor, ambient
    electrical = [] # 3 channels: current phases
    rul = []
    
    for _ in range(n_samples):
        total_life = np.random.randint(200, 400)
        current_age = np.random.randint(0, total_life - 20)
        wear = current_age / total_life
        
        t = np.arange(seq_length)
        
        # Vibration (high frequency, affected by wear)
        vib_data = np.zeros((seq_length, 2))
        freq = 25 + np.random.normal(0, 1)
        vib_data[:, 0] = (0.5 + 2 * wear) * np.sin(2 * np.pi * freq * t / 100)
        vib_data[:, 1] = (0.3 + 1.5 * wear) * np.sin(2 * np.pi * freq * t / 100 + 0.5)
        vib_data += np.random.normal(0, 0.1, vib_data.shape)
        
        # Thermal (slow trends)
        therm_data = np.zeros((seq_length, 3))
        therm_data[:, 0] = 50 + 30 * wear + np.random.normal(0, 2, seq_length)  # Bearing
        therm_data[:, 1] = 60 + 20 * wear + np.random.normal(0, 2, seq_length)  # Motor
        therm_data[:, 2] = 25 + 5 * np.sin(t / 50) + np.random.normal(0, 1, seq_length)  # Ambient
        
        # Electrical (current patterns)
        elec_data = np.zeros((seq_length, 3))
        phase_shift = 2 * np.pi / 3
        for p in range(3):
            elec_data[:, p] = (10 + 3 * wear) * np.sin(2 * np.pi * 50 * t / 1000 + p * phase_shift)
            # Add harmonics with wear
            elec_data[:, p] += 0.5 * wear * np.sin(2 * np.pi * 150 * t / 1000 + p * phase_shift)
        elec_data += np.random.normal(0, 0.2, elec_data.shape)
        
        vibration.append(vib_data)
        thermal.append(therm_data)
        electrical.append(elec_data)
        rul.append(total_life - current_age)
    
    return (
        np.array(vibration),
        np.array(thermal),
        np.array(electrical),
        np.array(rul)
    )

print("Generating multi-modal data...")
vib_data, therm_data, elec_data, rul_data = generate_multimodal_data(n_samples=800)
print(f"Vibration: {vib_data.shape}")
print(f"Thermal: {therm_data.shape}")
print(f"Electrical: {elec_data.shape}")

In [ ]:
if HAS_TF:
    # Normalize each modality
    vib_scaled = (vib_data - vib_data.mean()) / vib_data.std()
    therm_scaled = (therm_data - therm_data.mean()) / therm_data.std()
    elec_scaled = (elec_data - elec_data.mean()) / elec_data.std()
    rul_norm = rul_data / rul_data.max()
    
    # Split
    indices = np.arange(len(rul_data))
    train_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42)
    
    # Build model
    mm_model = build_multimodal_model(
        vibration_shape=(128, 2),
        thermal_shape=(128, 3),
        electrical_shape=(128, 3)
    )
    
    mm_model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss='mse',
        metrics=['mae']
    )
    
    mm_model.summary()

In [ ]:
if HAS_TF:
    print("Training multi-modal fusion model...")
    mm_history = mm_model.fit(
        [
            vib_scaled[train_idx],
            therm_scaled[train_idx],
            elec_scaled[train_idx]
        ],
        rul_norm[train_idx],
        validation_split=0.15,
        epochs=40,
        batch_size=32,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True)
        ],
        verbose=1
    )
    
    # Evaluate
    y_pred_mm = mm_model.predict([
        vib_scaled[test_idx],
        therm_scaled[test_idx],
        elec_scaled[test_idx]
    ]).flatten() * rul_data.max()
    
    y_true_mm = rul_data[test_idx]
    
    rmse_mm = np.sqrt(mean_squared_error(y_true_mm, y_pred_mm))
    r2_mm = r2_score(y_true_mm, y_pred_mm)
    
    print(f"\nMulti-Modal Fusion Results:")
    print(f"  RMSE: {rmse_mm:.2f} cycles")
    print(f"  R²:   {r2_mm:.4f}")

## 8. Model Comparison

In [ ]:
if HAS_TF:
    # Compare all models
    results = {
        'Model': ['FNO (Physics-Informed)', 'Ensemble (VAE+LSTM+Transformer)', 'Multi-Modal Fusion'],
        'RMSE': [rmse, rmse_ens, rmse_mm],
        'R²': [r2, r2_ens, r2_mm]
    }
    
    results_df = pd.DataFrame(results)
    print("\n" + "="*60)
    print("MODEL COMPARISON")
    print("="*60)
    print(results_df.to_string(index=False))
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    x = np.arange(len(results['Model']))
    
    axes[0].bar(x, results['RMSE'], color=['#2ecc71', '#3498db', '#9b59b6'])
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(results['Model'], rotation=15, ha='right')
    axes[0].set_ylabel('RMSE (cycles)')
    axes[0].set_title('Model Comparison: RMSE (lower is better)')
    
    axes[1].bar(x, results['R²'], color=['#2ecc71', '#3498db', '#9b59b6'])
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(results['Model'], rotation=15, ha='right')
    axes[1].set_ylabel('R²')
    axes[1].set_title('Model Comparison: R² (higher is better)')
    axes[1].set_ylim(0, 1)
    
    plt.tight_layout()
    plt.savefig(f'{MODEL_DIR}/model_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()

## 9. Save Models

In [ ]:
if HAS_TF:
    # Save models
    fno_model.save(f'{MODEL_DIR}/fno_physics_informed.keras')
    ensemble_model.save(f'{MODEL_DIR}/ensemble_vae_lstm_transformer.keras')
    mm_model.save(f'{MODEL_DIR}/multimodal_fusion.keras')
    agent.model.save(f'{MODEL_DIR}/dqn_maintenance_scheduler.keras')
    
    # Save metadata
    metadata = {
        'framework': 'Hybrid Deep Learning for Predictive Maintenance',
        'models': {
            'fno': {
                'file': 'fno_physics_informed.keras',
                'description': 'Fourier Neural Operator with physics constraints',
                'rmse': float(rmse),
                'r2': float(r2)
            },
            'ensemble': {
                'file': 'ensemble_vae_lstm_transformer.keras',
                'description': 'VAE + LSTM + Transformer ensemble',
                'rmse': float(rmse_ens),
                'r2': float(r2_ens)
            },
            'multimodal': {
                'file': 'multimodal_fusion.keras',
                'description': 'Multi-modal sensor fusion with cross-attention',
                'rmse': float(rmse_mm),
                'r2': float(r2_mm)
            },
            'rl_scheduler': {
                'file': 'dqn_maintenance_scheduler.keras',
                'description': 'DQN agent for maintenance scheduling',
                'test_failure_rate': float(test_failures / test_episodes)
            }
        },
        'trends_2025': [
            'Physics-informed neural networks',
            'Ensemble deep learning',
            'Multi-modal sensor fusion',
            'RL for maintenance optimization',
            'Uncertainty quantification'
        ]
    }
    
    with open(f'{MODEL_DIR}/hybrid_metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"\nModels saved to {MODEL_DIR}/")

## Summary

This notebook demonstrated **Hybrid Frameworks** for Predictive Maintenance:

### Models Implemented:

| Model | Key Innovation | Use Case |
|-------|----------------|----------|
| **FNO** | Frequency-domain processing | Periodic/vibration signals |
| **Ensemble** | VAE + LSTM + Transformer | Robust predictions |
| **Multi-Modal** | Cross-attention fusion | Multiple sensor types |
| **RL Scheduler** | DQN for decisions | Maintenance optimization |

### Key Concepts:

1. **Physics-Informed Learning**: Incorporate domain knowledge
2. **Ensemble Methods**: Combine multiple architectures
3. **Multi-Modal Fusion**: Specialize processing per sensor type
4. **End-to-End Optimization**: Prediction → Decision pipeline

### When to Use Hybrid Approaches:

- Complex systems with multiple sensor types
- Need for robust, explainable predictions
- Integration with decision-making systems
- Known physics to incorporate